In [1]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row
movie_rating = sc.textFile("/FileStore/tables/ratings.csv")

In [2]:
header = movie_rating.take(1)[0]
rating_data = movie_rating.filter(lambda line: line!=header).map(lambda line: line.split(",")).map(lambda p: Row(userId=int(p[0]), movieId=int(p[1]), rating=float(p[2]))).cache() 
ratingdata = spark.createDataFrame(rating_data)
ratingdata.show()

+-------+------+------+
movieId|rating|userId|
+-------+------+------+
 1| 4.0| 1|
 3| 4.0| 1|
 6| 4.0| 1|
 47| 5.0| 1|
 50| 5.0| 1|
 70| 3.0| 1|
 101| 5.0| 1|
 110| 4.0| 1|
 151| 5.0| 1|
 157| 5.0| 1|
 163| 5.0| 1|
 216| 5.0| 1|
 223| 3.0| 1|
 231| 5.0| 1|
 235| 4.0| 1|
 260| 5.0| 1|
 296| 3.0| 1|
 316| 3.0| 1|
 333| 5.0| 1|
 349| 4.0| 1|
+-------+------+------+
only showing top 20 rows

In [3]:
(training, validation, test) = ratingdata.randomSplit([0.6, 0.2, 0.2])
training.show()

+-------+------+------+
movieId|rating|userId|
+-------+------+------+
 1| 2.0| 132|
 1| 2.0| 193|
 1| 2.0| 298|
 1| 2.5| 15|
 1| 2.5| 68|
 1| 3.0| 27|
 1| 3.0| 32|
 1| 3.0| 33|
 1| 3.0| 44|
 1| 3.0| 93|
 1| 3.0| 130|
 1| 3.0| 134|
 1| 3.0| 155|
 1| 3.0| 216|
 1| 3.0| 233|
 1| 3.0| 293|
 1| 3.0| 314|
 1| 3.5| 18|
 1| 3.5| 21|
 1| 3.5| 119|
+-------+------+------+
only showing top 20 rows

In [4]:
# # Build the recommendation model using ALS on the training data
# # Note we set cold start strategy to 'drop' to ensure we don't get NaN evaluation metrics
# als = ALS(maxIter=5, regParam=0.01, userCol="userId", itemCol="movieId", ratingCol="rating",
#           coldStartStrategy="drop")
# model = als.fit(training)

# # Evaluate the model by computing the RMSE on the test data
# predictions = model.transform(test)
# evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",
#                                 predictionCol="prediction")
# rmse = evaluator.evaluate(predictions)
# print("Root-mean-square error = " + str(rmse))

Root-mean-square error = 1.1887124301210041

In [5]:
def train_ALS(train_data, validation_data, num_iters, reg_param, ranks):
    min_error = float('inf')
    best_rank = -1
    best_regularization = 0
    best_model = None
    for rank in ranks:
        for reg in reg_param:
            # write your approach to train ALS model
            als = ALS(rank=rank, maxIter=num_iters, regParam=reg, userCol="userId", itemCol="movieId", ratingCol="rating", coldStartStrategy="drop")
            model = als.fit(train_data)
            # make prediction
            predictions = model.transform(validation_data)
            # get the rating result
            evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction")
            # get the RMSE
            rmse = evaluator.evaluate(predictions)
            error = rmse
            print ('{} latent factors and regularization = {}: validation RMSE is {}'.format(rank, reg, error))
            if error < min_error:
                min_error = error
                best_rank = rank
                best_regularization = reg
                best_model = model
    print ('\nThe best model has {} latent factors and regularization = {}'.format(best_rank, best_regularization))
    return best_model

In [6]:
#find best parameters with training and validation data
num_iterations = 10
ranks = [6, 8, 10, 12, 14]
reg_params = [0.05, 0.1, 0.2, 0.4, 0.8]

import time
start_time = time.time()
final_model = train_ALS(training, validation, num_iterations, reg_params, ranks)

print ('Total Runtime: {:.2f} seconds'.format(time.time() - start_time))

6 latent factors and regularization = 0.05: validation RMSE is 0.9685173738791263
6 latent factors and regularization = 0.1: validation RMSE is 0.9087497575585733
6 latent factors and regularization = 0.2: validation RMSE is 0.8934464614009568
6 latent factors and regularization = 0.4: validation RMSE is 0.963354059286928
6 latent factors and regularization = 0.8: validation RMSE is 1.1804709309193921
8 latent factors and regularization = 0.05: validation RMSE is 0.9823873540104197
8 latent factors and regularization = 0.1: validation RMSE is 0.9137660728191023
8 latent factors and regularization = 0.2: validation RMSE is 0.8957780631942163
8 latent factors and regularization = 0.4: validation RMSE is 0.9640562103669712
8 latent factors and regularization = 0.8: validation RMSE is 1.1804614835394396
10 latent factors and regularization = 0.05: validation RMSE is 0.9915443248588652
10 latent factors and regularization = 0.1: validation RMSE is 0.9110211843546103
10 latent factors and regularization = 0.2: validation RMSE is 0.8934590506811548
10 latent factors and regularization = 0.4: validation RMSE is 0.9640436152069735
10 latent factors and regularization = 0.8: validation RMSE is 1.1804573973820676
12 latent factors and regularization = 0.05: validation RMSE is 0.996995988491446
12 latent factors and regularization = 0.1: validation RMSE is 0.9096793168828271
12 latent factors and regularization = 0.2: validation RMSE is 0.8934654805098203
12 latent factors and regularization = 0.4: validation RMSE is 0.9643276931964784
12 latent factors and regularization = 0.8: validation RMSE is 1.180459351505263
14 latent factors and regularization = 0.05: validation RMSE is 1.0020578166779732
14 latent factors and regularization = 0.1: validation RMSE is 0.9108260841053488
14 latent factors and regularization = 0.2: validation RMSE is 0.8938655859643321
14 latent factors and regularization = 0.4: validation RMSE is 0.9641298040182701
14 latent factors and regularization = 0.8: validation RMSE is 1.1804534375079454

The best model has 6 latent factors and regularization = 0.2
Total Runtime: 683.95 seconds

In [7]:
#test
als = ALS(rank=6, maxIter=10, regParam=0.2, userCol="userId", itemCol="movieId", ratingCol="rating", coldStartStrategy="drop")
model = als.fit(training)
predictions = model.transform(validation)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",
                                predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

Root-mean-square error = 0.8934464614009568

In [8]:
# Generate top 10 movie recommendations for each user
userRecs = model.recommendForAllUsers(10)
userRecs.show()
# Generate top 10 user recommendations for each movie
movieRecs = model.recommendForAllItems(10)



+------+--------------------+
userId| recommendations|
+------+--------------------+
 471|[[6818, 4.82872],...|
 463|[[112804, 5.65140...|
 496|[[6818, 4.875598]...|
 148|[[112804, 5.26278...|
 540|[[112804, 5.74922...|
 392|[[6818, 5.5200763...|
 243|[[112804, 6.99352...|
 31|[[112804, 5.93027...|
 516|[[112804, 5.20674...|
 580|[[112804, 5.21783...|
 251|[[112804, 6.26222...|
 451|[[112804, 5.39295...|
 85|[[6818, 4.8489704...|
 137|[[141718, 5.16271...|
 65|[[112804, 5.44737...|
 458|[[112804, 6.06546...|
 481|[[6818, 4.298811]...|
 53|[[112804, 7.57660...|
 255|[[141718, 4.11802...|
 588|[[112804, 5.07934...|
+------+--------------------+
only showing top 20 rows

In [9]:
# Generate top 10 movie recommendations for a specified set of users
users = ratingdata.select(als.getUserCol()).distinct().limit(3)
userSubsetRecs = model.recommendForUserSubset(users, 10)
userSubsetRecs.show()
# Generate top 10 user recommendations for a specified set of movies
movies = ratingdata.select(als.getItemCol()).distinct().limit(3)
movieSubSetRecs = model.recommendForItemSubset(movies, 10)

+------+--------------------+
userId| recommendations|
+------+--------------------+
 26|[[112804, 4.85825...|
 474|[[112804, 4.94617...|
 29|[[112804, 5.63649...|
+------+--------------------+